------------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import pandas as pd
import sys
sys.path.append('/home/avic/Rotation-Generalization')
from tools import *

In [ ]:
exp_path = '/home/avic/Rotation-Generalization/exps3.csv'

In [ ]:
d = pd.read_csv(exp_path, index_col=0)

In [ ]:
# def get_unrestricted(exp):
#     if exp.restriction_axes == '(0, 1)':
#         return 2
#     if exp.restriction_axes == '(0, 2)':
#         return 1
#     return 0

In [ ]:
# d['unrestricted_axis'] = d.apply(get_unrestricted, axis=1)

In [ ]:
d = d.drop(columns=['eval_heatmap_id_scaled', 'eval_heatmap_ood_scaled', 'eval_heatmap_id'])

In [ ]:
d['eval_heatmap_scaled'] = d.apply(lambda exp: os.path.join(exp.eval_dir, f'{exp["name"]}_heatmap_scaled.npy'), axis=1)
d['eval_heatmap_id'] = d.apply(lambda exp: os.path.join(exp.eval_dir, f'{exp["name"]}_heatmap_id.npy'), axis=1)

In [ ]:
d

In [ ]:
# d['eval_heatmap_id_scaled'] = d.apply(lambda exp: os.path.join(exp.eval_dir, f'{exp["name"]}_heatmap_id_scaled.npy'), axis=1)
# d['eval_heatmap_ood_scaled'] = d.apply(lambda exp: os.path.join(exp.eval_dir, f'{exp["name"]}_heatmap_ood_scaled.npy'), axis=1)

In [ ]:
d.to_csv(exp_path)

------------------

In [ ]:
import pandas as pd
import os
import shutil

In [ ]:
def remove_exp(exp):
    os.remove(exp.logs)
    os.remove(exp.eval)
    shutil.rmtree(exp.tensorboard_logs_dir)
    os.remove(exp.testing_frame_path)
    os.remove(exp.stats)
    os.remove(exp.checkpoint)
#     os.remove(exp.eval_heatmap)
    os.remove(exp.eval_heatmap_id)
    os.remove(exp.eval_heatmap_ood)

In [ ]:
d = pd.read_csv('/home/avic/Rotation-Generalization/exps2.csv')

In [ ]:
subset = d[(d.model_type == 'ModelType.ResNet') & (~d.augment) & (d.pretrained) & (~d.scale) & (d.training_category == 'SM')]

In [ ]:
subset.job_id

In [ ]:
for exp in subset.iterrows():
    remove_exp(exp[1])

---------------------------------

In [ ]:
import pandas as pd
import os
import tqdm
import subprocess
import sys
sys.path.append('/home/avic/Rotation-Generalization')
from dataset_path import *

In [ ]:
files = [set(), set(), set(), set(), set()]

In [ ]:
for i, dataset_i in enumerate([1550, 1600, 1650, 1700,1950]):
    path = DatasetPath.get_dataset(dataset_i // 50)
    for file in os.listdir(path.image_path):
        files[i].add(file)

In [ ]:
arg_list = []
for i, dataset_i in enumerate([1550, 1600, 1650, 1700,1950]):
    path = DatasetPath.get_dataset(dataset_i // 50)
    for j, annotation_path in enumerate(path.annotations_paths()):
        d = pd.read_csv(annotation_path)
        for row in d.itertuples():
            if row.image_name.split('/')[-1] not in files[i]:
                
                arg_list.append((dataset_i+j, annotation_path, row.Index))

In [ ]:
arg_list

In [ ]:
my_env = os.environ.copy()
my_env["CUDA_VISIBLE_DEVICES"] = "6"
for arg_item in tqdm.tqdm(arg_list):
    a = [
        '/home/avic/blender',
        '-b',
        '-noaudio',
        '-P',
        '/home/avic/Rotation-Generalization/render/rerender_image.py',
        str(arg_item[0]),
        arg_item[1],
        str(arg_item[2])
    ]
#     print(subprocess.run(f'/home/avic/blender -b -noaudio -P /home/avic/Rotation-Generalization/render/rerender_image.py {arg_item[0]} {arg_item[1]} {arg_item[2]}', env=my_env))
    result = subprocess.run(a, env=my_env, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))
    print(result.stderr.decode('utf-8'))